In [4]:
from transformers import cached_download
cached_download.clean()


ImportError: cannot import name 'cached_download' from 'transformers' (c:\Users\khali\anaconda3\envs\trader\lib\site-packages\transformers\__init__.py)

In [1]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset



In [6]:
from transformers import LlamaTokenizer, LlamaForCausalLM

model_name = "meta-llama/Meta-Llama-3-70B"

tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

# Load your dataset here


OSError: Can't load tokenizer for 'meta-llama/Meta-Llama-3-70B'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Meta-Llama-3-70B' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [7]:
# Load the pre-trained Llama 3 model and tokenizer
model_name = "meta-llama/Meta-Llama-3-8B"  # Update with the exact model path if needed
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

# Load the ashraq/financial-news-articles dataset
dataset = load_dataset("ashraq/financial-news-articles")

# Split dataset into train and test sets
train_dataset = dataset['train']
test_dataset = dataset['test']

# Preprocess the data
instruction = ("You are a summarizer and your job is to summarize whatever user asks you to in the most concise manner while being safe. "
               "Your summaries must have the same meaning and not include any false information. Make sure you don't use any external knowledge "
               "other than what's provided to you. Your final output must only be the summarized text.")



OSError: Can't load tokenizer for 'meta-llama/Meta-Llama-3-8B'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'meta-llama/Meta-Llama-3-8B' is the correct path to a directory containing all relevant files for a LlamaTokenizer tokenizer.

In [ ]:
def preprocess_function(examples):
    inputs = [f"{instruction} {doc}" for doc in examples['text']]  # Assuming 'text' field contains the articles
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=150, truncation=True)  # Assuming 'summary' field contains the summaries
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    save_steps=10_000,
    logging_steps=2_000,
    fp16=True,  # Enable mixed precision training if supported
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()



In [ ]:
# Save the model
model.save_pretrained("./fine-tuned-llama-3-news-summary")
tokenizer.save_pretrained("./fine-tuned-llama-3-news-summary")
